# Laboratorio Redes Neuronales (parte 2)

En este laboratorio vamos a ver algunos ejemplos más de uso de Keras para resolver problemas con redes neuronales.

Primero vamos a ver un módulo, Keras Tuner ([link](https://keras.io/guides/keras_tuner/getting_started/)) que permite buscar, mediante diferentes métodos los mejores hiperparámetros, una tarea bastante tediosa.

Vemos que la búsqueda de hiperparámetros puede ser un poco tediosa. Para ello keras provee Keras Tuner , que simplifica la búsqueda manual, mediante diferentes métodos de búsqueda, que también pueden elegirse.

In [2]:
import numpy as np
from tensorflow import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, x_test.shape

11490434/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (10000, 28, 28))

Normalizamos...

In [4]:
# Utilizando la operacion reshape, convertir las imagenes de matriz a vector
x_train1 = x_train.reshape(60000,28*28)
x_test1 = x_test.reshape(10000,28*28)
# Convertir el tipo de los datos de tipo np.uint8 a np.float32
x_train1 = x_train1.astype(np.float32)
x_test1 = x_test1.astype(np.float32)
# Dividimos el valor de cada pixel entre 255. para asegurarnos que todos los pixeles queden en el rango $[0,1]$
x_train1 = x_train1 / 255.0
x_test1 = x_test1 / 255.0
# Restamos a cada pixel 0.5 para asegurarnos que todos los pixeles queden en el rango $[-0.5,0.5]$
x_train1 = x_train1 - 0.5
x_test1 = x_test1 - 0.5
####################################################################################################################


In [5]:
# Cuando verificamos que quedó bien, modificamos los datos originales
x_train = x_train1
x_test = x_test1

Separamos entrenamiento y validación

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=0)

# Verificamos que todo este bien
assert len(x_train) == len(y_train)
assert len(x_val) == len(y_val)

Primero instalamos Keras Tuner...

In [7]:
!pip install keras-tuner --upgrade
import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 25.0 MB/s eta 0:00:00
Using TensorFlow backend


Supongamos que queremos probar una red como la que trabajamos en la clase anterior para reconocer dígitos, y nos interesa saber si funciona mejor una relu o una sigmoid.

Para utilizar Keras Tuner vamos primero a crear una función que define el modelo, recibiendo los hiperparámetros como argumento (en este caso,  definimos que la función de activación admite dos valores, y que la cantidad de unidades de la capa puede ser entre 1 y 32):



In [8]:
def build_model(hp):
  activation1 = hp.Choice('activation1',['relu','sigmoid'] )
  units1 = hp.Int("units", min_value=1, max_value=512, step=1)

  model = keras.Sequential()
  model.add(keras.layers.Dense(input_dim = (x_train.shape[1]), units = units1, activation = activation1))
  model.add(keras.layers.Dense(10, activation="softmax", name="output_layer"))

  model.compile(optimizer = keras.optimizers.SGD(0.001),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy']
                )
  return model

In [9]:
build_model(kt.HyperParameters())


Vamos a definir luego el tuner, que permite probar diferentes modelos y elegir el mejor. Para eso, le vamos a decir que utilice como objetivo a minimizar la pérdida en el conjunto de validación, y un máximo de 5 intentos, a partir del modelo parametrizado que definimos en el paso anterior. El tuner que vamos a utilizar es una búsqueda aleatoria (Keras Tuner ofrece otras opciones)

In [10]:
tuner = kt.RandomSearch(build_model, objective='val_loss', max_trials=5, overwrite=True,
    directory="my_dir")
tuner.search_space_summary()

Search space summary
Default search space size: 2
activation1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 512, 'step': 1, 'sampling': 'linear'}


Probamos con el tuner un aprendizaje con 5 épocas (observar que no llamamos a compile ni a try)

In [11]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
best_model = tuner.get_best_models()[0]

tuner.results_summary()


Trial 5 Complete [00h 00m 42s]
val_loss: 0.4717792868614197

Best val_loss So Far: 0.4491737186908722
Total elapsed time: 00h 04m 12s
Results summary
Results in my_dir/untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 2 summary
Hyperparameters:
activation1: relu
units: 510
Score: 0.4491737186908722

Trial 3 summary
Hyperparameters:
activation1: relu
units: 470
Score: 0.44944360852241516

Trial 4 summary
Hyperparameters:
activation1: relu
units: 180
Score: 0.4717792868614197

Trial 0 summary
Hyperparameters:
activation1: relu
units: 123
Score: 0.47789862751960754

Trial 1 summary
Hyperparameters:
activation1: sigmoid
units: 467
Score: 1.2568233013153076


Vemos el resultado de nuestro mejor modelo contra el conjunto de testeo

In [12]:
predictionstest = best_model.predict(x_test)
y_test_pred = np.argmax(predictionstest, axis=1)
print(classification_report(y_test, y_test_pred))

313/313 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       980
           1       0.92      0.96      0.94      1135
           2       0.88      0.83      0.86      1032
           3       0.88      0.88      0.88      1010
           4       0.86      0.90      0.88       982
           5       0.86      0.81      0.83       892
           6       0.89      0.92      0.91       958
           7       0.91      0.88      0.89      1028
           8       0.85      0.84      0.85       974
           9       0.87      0.86      0.86      1009

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



*Se pide*: Implementar una búsqueda de hiperparámetros con RandomSearch, utilizando:

*   Red de dos capas ocultas
*   Optimizador SGD
*   Entrenamiento en la búsqueda de 10 épocas
*   Máximo de intentos: 20

Variar:

*   Learning rate de SGD entre 1e-4 y 1e-2 (valor real)
*   funciones de activación de capas ocultas: ['relu', 'sigmoid']
*   Cantidad de neuronas de las capas: [8, 32, 64]

Mostrar métricas para el mejor modelo obtenido

In [13]:
# FUNCIÓN CREADORA DE MODELO

from keras.layers import Dense
from keras.optimizers import SGD

def build_model_ejercicio(hp):

  # Hiperparámetros
  lr = hp.Float('lr', min_value = 1e-4, max_value = 1e-2, sampling = 'log')
  activation = hp.Choice('activation', ['relu', 'sigmoid'])
  units = hp.Choice('units', [8,32,64])

  # Modelo
  model = keras.Sequential(name = 'keras_tuner_model')
  model.add(Dense(units = units, activation = activation, input_dim = x_train.shape[1], name = 'hidden_layer_1'))
  model.add(Dense(units = units, activation = activation, name = 'hidden_layer_2'))
  model.add(Dense(units = 10, activation = 'softmax', name = 'output_layer'))

  # Compilación
  model.compile(
          optimizer = SGD(learning_rate = lr),
          loss = 'sparse_categorical_crossentropy',
          metrics = ['accuracy']
          )

  return model

In [14]:
# CREADOR DE TUNER
tuner_ejercicio = kt.RandomSearch(
                                  hypermodel = build_model_ejercicio,
                                  objective = 'val_loss',
                                  max_trials = 20,
                                  overwrite = True,
                                  directory = 'my_dir'
                                  )

tuner_ejercicio.search_space_summary()

Search space summary
Default search space size: 3
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units (Choice)
{'default': 8, 'conditions': [], 'values': [8, 32, 64], 'ordered': True}


In [15]:
# BÚSQUEDA
tuner_ejercicio.search(x_train, y_train, epochs = 10, validation_data = (x_val, y_val))
best_model_ejercicio = tuner_ejercicio.get_best_models()[0]

tuner_ejercicio.results_summary()

Trial 20 Complete [00h 01m 23s]
val_loss: 2.286397695541382

Best val_loss So Far: 0.19195345044136047
Total elapsed time: 00h 23m 46s
Results summary
Results in my_dir/untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 07 summary
Hyperparameters:
lr: 0.007487658008462713
activation: relu
units: 32
Score: 0.19195345044136047

Trial 04 summary
Hyperparameters:
lr: 0.002056895981466407
activation: relu
units: 64
Score: 0.27845072746276855

Trial 09 summary
Hyperparameters:
lr: 0.001606325857963324
activation: relu
units: 64
Score: 0.3017254173755646

Trial 13 summary
Hyperparameters:
lr: 0.0012306883948819032
activation: relu
units: 64
Score: 0.32284125685691833

Trial 14 summary
Hyperparameters:
lr: 0.002793838236735594
activation: relu
units: 8
Score: 0.3836863338947296

Trial 08 summary
Hyperparameters:
lr: 0.003999684359589883
activation: relu
units: 8
Score: 0.4334386885166168

Trial 10 summary
Hyperparameters:
lr: 0.000511281121262892
activat

In [17]:
predictionstest_ejercicio = best_model_ejercicio.predict(x_test)
y_test_pred_ejercicio = np.argmax(predictionstest_ejercicio, axis=1)
print(classification_report(y_true = y_test, y_pred = y_test_pred_ejercicio, digits = 3))

313/313 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0      0.954     0.982     0.968       980
           1      0.959     0.984     0.971      1135
           2      0.957     0.919     0.937      1032
           3      0.913     0.946     0.929      1010
           4      0.920     0.953     0.936       982
           5      0.958     0.902     0.930       892
           6      0.940     0.954     0.947       958
           7      0.968     0.921     0.944      1028
           8      0.901     0.927     0.914       974
           9      0.933     0.907     0.920      1009

    accuracy                          0.940     10000
   macro avg      0.940     0.939     0.940     10000
weighted avg      0.941     0.940     0.940     10000

